In [22]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster

data = pd.read_csv('cloud-data/bus_data_modified.csv')
# convert timestamp to datetime
data['timestamp'] = pd.to_datetime(data['timestamp'])
# shift all timestamps 6 hrs back
data['timestamp'] = data['timestamp'] - pd.Timedelta(hours=4)
# only use the latitude and longitude columns between certain values
data = data[(data['latitude'] > 40.4) & (data['latitude'] < 40.53)]
data = data[(data['longitude'] > -74.5) & (data['longitude'] < -74.3)]
data.head()

,id,name,type,calculatedCourse,routeName,created,longitude,latitude,paxLoad,totalCap,more,deviceId,routeBlockId,timestamp,load,stop
0,4855,4131,Transit Bus,212.735226,LX Route,02:23 PM,-74.433571,40.519749,0,67,101,402213,111417,2024-10-31 14:24:00,0.000000,Quads
1,4859,4177,Transit Bus,128.232575,LX Route,02:23 PM,-74.436304,40.524327,27,67,102,402325,111417,2024-10-31 14:24:00,0.402985,Livingston Student Center
2,4873,4191,Transit Bus,240.802514,B Route,02:23 PM,-74.457712,40.523463,8,67,101,402329,113278,2024-10-31 14:24:00,0.119403,NaN
3,4876,4194,Transit Bus,299.246496,B Route,02:23 PM,-74.438946,40.521624,4,67,102,402330,113250,2024-10-31 14:24:00,0.059701,NaN
4,17624,4002,Heavy Duty Transit LF,226.694476,REXB Route,02:23 PM,-74.438830,40.483538,2,68,102,402643,112511,2024-10-31 14:24:00,0.029412,NaN


In [27]:
def get_home_stop(stops):
    for stop in stops:
        if type(stop) == float: continue
        if 'student center' in stop.lower():
            return stop
    
    if type(stops[0]) == float: return stops[1]
    return stops[0]

In [24]:
route = data['routeName'].unique()[1]
route

'B Route'

In [25]:
# create a data frame with unique timestamps as index and columns with the number of buses on a given route at the stop
data_route = data[data['routeName'] == route]

stops = data_route['stop'].unique()

timestamps = data_route['timestamp'].unique()


In [31]:

def create_csv_route(route):
    # create a data frame with unique timestamps as index and columns with the number of buses on a given route at the stop
    data_route = data[data['routeName'] == route]

    stops = data_route['stop'].unique()

    timestamps = data_route['timestamp'].unique()

    # convert timestamps to dataframe with stops as columns
    df = pd.DataFrame(index=timestamps, columns=stops)
    df.head()

    # Initialize the resulting DataFrame
    result_columns = ['bus_id', 'timestamp', 'load', 'stop', 'time_on_route', 'time_waiting', 'time_to_complete']
    result_data = []

    main_stop = get_home_stop(stops)

    # Process each bus on the route
    for bus_id in data_route['id'].unique():
        data_bus = data_route[data_route['id'] == bus_id].copy()

        if len(data_bus) <= 20: # Skip buses with less than 20 records
            continue

        # Initialize columns for time calculations
        data_bus['time_to_complete'] = 0
        data_bus['time_waiting'] = 0
        data_bus['time_on_route'] = 0

        for i in range(len(data_bus) - 2, 0, -1):
            time_diff = (data_bus['timestamp'].iloc[i+1] - data_bus['timestamp'].iloc[i]).seconds

            if time_diff > 300:  # Reset if time difference > 5 minutes
                data_bus.loc[data_bus.index[i], ['time_to_complete', 'time_waiting']] = 0
                continue

            if type(data_bus['stop'].iloc[i]) == str and data_bus['stop'].iloc[i] != 'NaN':
                data_bus.loc[data_bus.index[i], 'time_waiting'] = (
                    data_bus['time_waiting'].iloc[i+1] + time_diff
                )
                data_bus.loc[data_bus.index[i], 'time_to_complete'] = 0
            else:
                data_bus.loc[data_bus.index[i], 'time_to_complete'] = (
                    data_bus['time_to_complete'].iloc[i+1] + time_diff
                )
                data_bus.loc[data_bus.index[i], 'time_waiting'] = 0

            if data_bus['stop'].iloc[i] == main_stop:
                data_bus.loc[data_bus.index[i], 'time_on_route'] = 0
            else:
                data_bus.loc[data_bus.index[i], 'time_on_route'] = (
                    data_bus['time_on_route'].iloc[i+1] + time_diff
                )

        # Aggregate data for the bus
        for i, row in data_bus.iterrows():
            result_data.append([
                bus_id,
                row['timestamp'],
                row['load'],
                row['stop'],
                row['time_on_route'],
                row['time_waiting'],
                row['time_to_complete'],
            ])

    # Combine all buses into a single DataFrame
    result_df = pd.DataFrame(result_data, columns=result_columns)
    result_df.to_csv(f'output/{route.replace(" ", "_")}_stop.csv', index=False)
    # Display the first few rows of the combined DataFrame
    result_df.head()

create_csv_route(route)

In [ ]:
route

'Weekend 1'

In [32]:
for route in data['routeName'].unique():
    create_csv_route(route)

In [ ]:
result_df.describe()

NameError: name 'result_df' is not defined

In [ ]:
""" for bus_id in result_df['bus_id'].unique():
    bus_data = result_df[result_df['bus_id'] == bus_id]

    # Time on route plot
    plt.figure(figsize=(10, 5))
    plt.plot(bus_data['timestamp'], bus_data['time_on_route'] / 60, label=f'Bus {bus_id}')
    plt.xlabel('Stop')
    plt.ylabel('Time on Route (minutes)')
    plt.xticks(rotation=45)
    plt.title(f'Time to Complete Route for Bus {bus_id}')
    plt.legend()
    plt.show()

    # Time waiting plot
    plt.figure(figsize=(10, 5))
    plt.plot(bus_data['timestamp'], bus_data['time_waiting'] / 60, label=f'Bus {bus_id}')
    plt.xlabel('Stop')
    plt.ylabel('Time Waiting at Stop (minutes)')
    plt.xticks(rotation=45)
    plt.title(f'Time Waiting at Stops for Bus {bus_id}')
    plt.legend()
    plt.show() """

" for bus_id in result_df['bus_id'].unique():\n    bus_data = result_df[result_df['bus_id'] == bus_id]\n\n    # Time on route plot\n    plt.figure(figsize=(10, 5))\n    plt.plot(bus_data['timestamp'], bus_data['time_on_route'] / 60, label=f'Bus {bus_id}')\n    plt.xlabel('Stop')\n    plt.ylabel('Time on Route (minutes)')\n    plt.xticks(rotation=45)\n    plt.title(f'Time to Complete Route for Bus {bus_id}')\n    plt.legend()\n    plt.show()\n\n    # Time waiting plot\n    plt.figure(figsize=(10, 5))\n    plt.plot(bus_data['timestamp'], bus_data['time_waiting'] / 60, label=f'Bus {bus_id}')\n    plt.xlabel('Stop')\n    plt.ylabel('Time Waiting at Stop (minutes)')\n    plt.xticks(rotation=45)\n    plt.title(f'Time Waiting at Stops for Bus {bus_id}')\n    plt.legend()\n    plt.show() "

In [ ]:
data_bus = data_route[data_route['id'] == 7179]
data_bus.tail()

,id,name,type,calculatedCourse,routeName,created,longitude,latitude,paxLoad,totalCap,more,deviceId,routeBlockId,timestamp,load,stop


In [ ]:
len(data_bus)

0

In [ ]:
data_bus['stop'].unique()

array([], dtype=object)